# Loop 3 Analysis: Fix Overlap Detection

The LB uses `intersects() and not touches()` NOT `intersection.area > 0`.
These are DIFFERENT checks. Let's verify and fix.

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

# Tree geometry
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

BASE_TREE_VERTICES = np.array([
    [0.0, TIP_Y],
    [TOP_W/2, TIER_1_Y],
    [TOP_W/4, TIER_1_Y],
    [MID_W/2, TIER_2_Y],
    [MID_W/4, TIER_2_Y],
    [BASE_W/2, BASE_Y],
    [TRUNK_W/2, BASE_Y],
    [TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, BASE_Y],
    [-BASE_W/2, BASE_Y],
    [-MID_W/4, TIER_2_Y],
    [-MID_W/2, TIER_2_Y],
    [-TOP_W/4, TIER_1_Y],
    [-TOP_W/2, TIER_1_Y],
])

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    rotated = BASE_TREE_VERTICES @ rotation_matrix.T
    translated = rotated + np.array([x, y])
    return Polygon(translated)

def parse_submission(df):
    result = df.copy()
    for col in ['x', 'y', 'deg']:
        result[col] = result[col].str.replace('s', '').astype(float)
    return result

print('Functions defined')

Functions defined


In [2]:
# Define BOTH overlap checks to compare
def check_overlaps_area_based(polygons):
    """OLD check: intersection.area > 0"""
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            intersection = polygons[i].intersection(polygons[j])
            if intersection.area > 0:
                return True, (i, j), intersection.area
    return False, None, 0

def check_overlaps_lb_style(polygons):
    """LB check: intersects() and not touches()"""
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                return True, (i, j)
    return False, None

print('Both overlap checks defined')

Both overlap checks defined


In [3]:
# Load current submission and check group 008 specifically
print('Loading current submission...')
df = pd.read_csv('/home/submission/submission.csv')
parsed = parse_submission(df)

# Check group 008
config_df = parsed[parsed['id'].str.startswith('008_')]
print(f'Group 008 has {len(config_df)} trees')
print(config_df)

polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]

# Check with BOTH methods
area_overlap, area_pair, area_val = check_overlaps_area_based(polygons)
lb_overlap, lb_pair = check_overlaps_lb_style(polygons)

print(f'\nArea-based check: overlap={area_overlap}, pair={area_pair}, area={area_val}')
print(f'LB-style check: overlap={lb_overlap}, pair={lb_pair}')

Loading current submission...
Group 008 has 8 trees
       id         x         y         deg
28  008_0 -0.249562 -0.411092   51.766738
29  008_1  0.664668 -0.857305  113.629378
30  008_2 -0.664668  0.257305  293.629378
31  008_3  0.557305  0.326837  203.629378
32  008_4  0.249562 -0.188908  231.766738
33  008_5 -0.206589 -0.855262  293.629378
34  008_6  0.206589  0.255262  113.629378
35  008_7 -0.557305 -0.926837   23.629378

Area-based check: overlap=True, pair=(1, 5), area=6.162975822039155e-32
LB-style check: overlap=True, pair=(1, 5)


In [4]:
# Check ALL groups with BOTH methods
print('Checking all groups with BOTH methods...')

area_overlaps = []
lb_overlaps = []

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = parsed[parsed['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    
    area_has, _, _ = check_overlaps_area_based(polygons)
    lb_has, _ = check_overlaps_lb_style(polygons)
    
    if area_has:
        area_overlaps.append(n)
    if lb_has:
        lb_overlaps.append(n)

print(f'Area-based overlaps: {len(area_overlaps)} groups')
print(f'LB-style overlaps: {len(lb_overlaps)} groups')

# Find groups that differ
only_lb = set(lb_overlaps) - set(area_overlaps)
only_area = set(area_overlaps) - set(lb_overlaps)
print(f'\nGroups with LB overlap but NOT area overlap: {sorted(only_lb)}')
print(f'Groups with area overlap but NOT LB overlap: {sorted(only_area)}')

Checking all groups with BOTH methods...


Area-based overlaps: 31 groups
LB-style overlaps: 33 groups

Groups with LB overlap but NOT area overlap: [40, 71]
Groups with area overlap but NOT LB overlap: []


In [ ]:
# Detailed analysis of group 008
print('Detailed analysis of group 008...')
config_df = parsed[parsed['id'].str.startswith('008_')]
polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]

for i in range(len(polygons)):
    for j in range(i+1, len(polygons)):
        intersects = polygons[i].intersects(polygons[j])
        touches = polygons[i].touches(polygons[j])
        intersection = polygons[i].intersection(polygons[j])
        
        if intersects:
            print(f'Trees {i} and {j}:')
            print(f'  intersects={intersects}, touches={touches}')
            print(f'  intersection type: {intersection.geom_type}')
            print(f'  intersection area: {intersection.area}')
            print(f'  LB would flag: {intersects and not touches}')